# Week 3-I: Data Preparation on the Kings County Housing Dataset

## Setup

In [ ]:
# Python ≥3.9 is required
import sys
assert sys.version_info >= (3, 9)

# Scikit-Learn ≥1.0 is required
import sklearn
assert sklearn.__version__ >= "1.0"

# Common imports
import numpy as np
import pandas as pd
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Precision options
np.set_printoptions(precision=2)
pd.options.display.float_format = '{:.3f}'.format

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## 1. Get the Data + Train/test split (again)

In [ ]:
housing = pd.read_csv(
    "../datasets/kings_county_house_data.csv",
    dtype={'zipcode': str}   # US ZIP codes look like numbers but we want to treat them like strings
)

Perform a stratified split wrt `sqft_living`:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

housing["sqft_living_cat"] = pd.cut(
    housing.sqft_living, 
    bins=[0., 1000., 2000., 3000., 4000., np.inf],
    labels=[1, 2, 3, 4, 5]
)
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in splitter.split(housing, housing.sqft_living_cat):
    train_set = housing.loc[train_index]
    test_set = housing.loc[test_index]
# delete the "sqft_living_cat"columns  
for set_ in (train_set, test_set):
    set_.drop("sqft_living_cat", axis=1, inplace=True)

## 2. Data Preparation Plan

From the EDA we have defined the following data preparation plan:
    
* We have identified a cutoff at latitude  ~47.5 between more expensive houses and cheaper houses.We will create a binary engineered feature to capture this. We could also create a cutoff at -126.1 long to separate the urban west from the rural east of the county. We will then remove `lat` and `long`.
* We have decided to discard `sqft_living15` and `sqft_living_above` in favour of `sqft_living`
* We have decided to add a binary engineered feature that indicates whether a house has a basement or not. We will then remove the continuous variable `sqft_basement`
* We will create a renovated binary flag. If a house is older than 25 years (relative to the most recent data in the dataset) and has not been renovated we will set renovated to 0, otherwise to 1. We will then remove the continuous variable `yr_built` and `yr__renovated`
* We have decided to collapse the 70 zipcodes into 9 zipcode groups based on average house prices in the zipcodes. This will be performed by `make_zipcode_groups()` function and you do not have to worry about the implementation details of it.
* Some houses report 0 bathrooms. We need to replace those values with more meaningful estimates.
* One house has 33 bedrooms. We will replace that value with 3, as it looks like a reporting mistake.   

## 3 Data preparation: check there are no missing values

We can use pd.DataFrame.isna() or pd.DataFrame.isnull() to look for null or missing values in any of our variables/features.

NOTE: axis=1 performs the operation along the columns

In [ ]:
## Look for rows with incomplete values
incomplete_rows = train_set[train_set.isna().any(axis=1)]
incomplete_rows

There are no missing values in our dataset.

## Feature Engineering

### **Exercise:** fill the missing feature engineering steps in the `engineer_features()` function. 

In [ ]:
from kings_county_utils import make_zipcode_groups, assign_to_zipcode_group

def engineer_features(data: pd.DataFrame) -> pd.DataFrame:
    # let's make a copy of the original dataset
    engineered_data = data.copy()    
    # create a variable "north_loc", 1 for latitudes greather than 47.5
    engineered_data["north_loc"] = np.where(engineered_data["lat"] > 47.5, 1, 0)
    # create a variable "rural_east", 1 for longitudes greater than -121.6
    ...
    # drop "lat" and "long"
    engineered_data = engineered_data.drop(columns=["lat", "long"])
    # drop "sqft_living15" and "sqft_living_above"
    engineered_data = engineered_data.drop(columns=["sqft_living15", "sqft_above"])
    # add a binary variable for the presence of a  basement, 1 if true 0 otherwise
    ...
    # drop "sqft_basement"
    engineered_data = engineered_data.drop(columns=["sqft_basement"])
    # create an "is_renovated" binary variable
    max_year = max(engineered_data["yr_built"].max(), engineered_data["yr_renovated"].max())
    engineered_data["renovated"] = np.where(
        (engineered_data["yr_built"] > max_year - 25) | (engineered_data["yr_renovated"] > 0), 1, 0
    )
    # drop "yr_built" and "yr_renovated"
    engineered_data = engineered_data.drop(columns=["yr_built", "yr_renovated"])
    # group zipcodes into groups (as we did last week)
    zipcode_groups = make_zipcode_groups(engineered_data)
    engineered_data["zipcode_group"] = engineered_data["zipcode"].apply(assign_to_zipcode_group, zipcode_groups=zipcode_groups)
    # drop "zipcode"
    engineered_data = engineered_data.drop(columns=["zipcode"])
    # replace 0 bathrooms with NaN (we will fill it later on)
    ...
    # replace 33 bedrooms with Nan (we will fill it later on)
    engineered_data.loc[engineered_data['bedrooms'] == 33, 'bedrooms'] = np.nan
    # drop id, date and price (as we won't use them)
    ...
    return engineered_data

In [ ]:
train_data = engineer_features(train_set)
train_labels = train_set["price"].copy()
train_data.sample(10, random_state=42)

In [ ]:
len(train_data.columns)

We now have 15 features in our dataset, but as we will see when preparing our data for feeding it into regression models we will get a few extra "dummy" features.

## 3 Check there are no missing values in the engineered dataset

In [ ]:
## Look for rows with incomplete values
incomplete_rows = train_data[train_data.isna().any(axis=1)]
incomplete_rows

As we can see we now have 1 sample with missing bedrooms info and 7 samples with missing bathrooms info. We need to deal with this missing values. The problem of dealing with missing values is called imputation.

### 3.1 Imputation

In statistics, imputation is the process of replacing missing data with substituted values.
In scikit-learn we can use the `SimpleImputer` calss to perform univariate imputation on missing  values. Generally we wil want to replace missing numeric (quantitative) and ordinal values with the median value of that feature. For categorical features we may want to either use a "missing"/"unknown" category, use the mode, or drop the samples with missing values.

Let's fill in the missing ordinal values of our dataset:

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
train_data_num = train_data.select_dtypes(include=[np.number])
train_data_num

Basically of our fields, with the exception of `zipocode_group` is quantitative or ordinal (or a binary variable):

In [ ]:
imputer.fit(train_data_num)
imputer.statistics_

The `SimpleImputer.statistics_` property is just the median value for each column:

In [ ]:
train_data_num.median()

We can now fill the missing values by applying the `transform()` method of the imputer to `train_data_num`:

In [ ]:
train_data_arr = imputer.transform(train_data_num)
# imputer.transform return a NumPy array, we need to wrapp it back into a dataframe with column names and index
train_data_num_filled = pd.DataFrame(
    train_data_arr,
    columns=train_data_num.columns,
    index=train_data_num.index
)

We can now verify that there are no more rows with missing values in `train_data_num`:

In [ ]:
train_data_num_filled[train_data_num_filled.isna().any(axis=1)]

### 3.2 Handling Categorical Attributes

In general we can consider three types of text features: categorical, ordinal, and unstructured.
Unstructured text is more the subject of Natural Language Processing, hence we will not consider its processing/encoding at this stage (and we have no unstructured data). Ordinal data are text categories that imply and intrinsic order such as the set ("BAD", "AVERAGE", "GOOD", "VERY GOOD", "EXCELLENT"). 
These are generally encodes as integers ("BAD" => 0, "AVERAGE" => 1, "GOOD" => 2, "VERY GOOD" => 3, "EXCELLENT" => 4). These transformations can be handed with custom functions as above or using `sklearn.preprocessing.OrdinalEncoder`. In our case all the ordinal features are already expressed as numbers so we don't need to do anything with them.

To handle Categorical Attributes that are not ordinal, a common solution is to create one binary attribute per category. This is called one-hot encoding, because only one attribute will be equal to 1 (hot), while the others will be 0 (cold). The new attributes are sometimes called *dummy attributes*. Scikit-Learn provides a `sklearn.preprocessing.OneHotEncoder` class to convert categorical values into one-hot vectors.

In our case we have the "zipcode" attribute that can be considered as categorical. Each "zipcode" category should become a mutually exclusive dummy attribute

### <b>Exercise:</b> Use the `OneHotEncoder` class to encode each ZIP code as a separate category. Check the documentation for appropriate use of the `OneHotEncoder` transformer. What kind of output do you get? 

In [ ]:
## Write your solution here
...

Notice that the output is a SciPy sparse matrix, instead of a NumPy array. This is very useful when you have categorical attributes with thousands of categories. After one-hot encoding, we get a matrix with thousands of columns, and the matrix is full of 0s except for a single 1 per row. We can get a dense array out of sparse matrix by calling the `.toarray()` method.

In [ ]:
train_data_cat_1hot.toarray()

In [ ]:
train_data_cat_1hot.toarray().shape

We can get the "column" names by checking the `OneHotEncoder.categories_` attribute:

In [ ]:
cat_encoder.categories_

### 3.3 Custom Transformers

You can define you own transformers creating a class that inherits both from `BaseEstimator` (so that it inherits the `.fit()` method) and the mixin class `TransformerMixin` (so that it acquires the `.tranform()` method)


### **Exercise:** convert `engineering_features()` to a scikit-learn's transformer subclassing `BaseEstimator` and `TransformerMixin`. I am already providing you with the backbone of the class and the `.fit()` method for you here below. 

NB: `data` won't have the "price" column as it is passed as `labels`


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineeringTransformer(BaseEstimator, TransformerMixin):
    def fit(self, data, labels=None) -> "FeatureEngineeringTransformer":
        self.zipcode_groups = make_zipcode_groups(pd.concat([data["zipcode"], labels], axis=1))
        return self

    def transform(self, data, labels=None) -> pd.DataFrame:
        pass # replace this line with your implementation


If your implementation is correct the cell below will run successfully:

In [ ]:
fe_trf = FeatureEngineeringTransformer()
fe_trf.fit(train_set.drop(
    columns=["price"]),      # training set data
    train_set["price"]       # training set labels
)
fe_trf.transform(train_set.drop(columns=["price"]))

## 4. Feature Scaling

One of the most important transformations you need to apply to your data is feature scaling. In the great majority of case, Machine Learning algorithms will not perform well when the input numerical attributes have very different scales.

There are two common ways to get all attributes to have the same scale:
* min-max scaling:  rescaling the range of features to scale the range in [0, 1] or [−1, 1] (using scikit-learn `MinMaxScaler`)
* standardization: scales the data to have zero mean and variance = 1 (using scikit-learn `StandardScaler`).

We will see an example of feature scaling below, when we'll show how all the preprocessing can be performed together building a pipeline.

In [ ]:
### Example courtesy of ChatGPT 
from sklearn.preprocessing import MinMaxScaler

ages = [[30], [40], [50], [60]]
scaler = MinMaxScaler()
scaled_ages = scaler.fit_transform(ages)

print("Original ages:")
print(ages)
print("\nScaled ages (min-max scaled):")
print(scaled_ages)

In [ ]:
### Example courtesy of ChatGPT 
from sklearn.preprocessing import StandardScaler

ages = [[30], [40], [50], [60]]
scaler = StandardScaler()
standardized_ages = scaler.fit_transform(ages)

print("Original ages:")
print(ages)
print("\nStandardized ages:")
print(standardized_ages)

## 5. Transformation Pipelines

Pipeline object are useful to chain transformations (and potentially estimators) together, ensuring clean code and reproducibility.

We are goin to use a `StandardScaler` directly after the `SimpleImputer` only for non-binary numerical features using a scikit-learn tranformation pipeline.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

cat_feats = ["zipcode_group"]
binary_feats = ["waterfront", "north_loc", "rural_east", "basement", "renovated"] 
num_feats = [
    el for el in list(
        train_data.select_dtypes(include=[np.number])
    ) if el not in binary_feats
]

In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])
train_data_num_scaled = num_pipeline.fit_transform(train_data[num_feats])

In [ ]:
train_data_num_scaled[:5]

As you can see the data has all been scaled to be zero-centred and with variance = 1.

Scikit-learn's built-in transformers output NumPy arrays by default. If we want to get a pandas data frame out, we can use the `set_output` API:

In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median").set_output(transform="pandas")),
    ('std_scaler', StandardScaler().set_output(transform="pandas"))
])
train_data_num_scaled = num_pipeline.fit_transform(train_data[num_feats])

In [ ]:
train_data_num_scaled.head()

### 6. Chaining all together - `ColumnTransformer` and `Pipeline`

So far we have transformed the data step by step, and given different transformations to different features in our dataset. Now, ideally, we would like to chain all the data preparation steps in a single operation. This would simplify applying the tranformations to different data sets (i.e. training and test set) as well as ensuring the reproducibility of our data manipulation pipeline (all the transformations would be executed atomically).

Until now, we have handled the categorical, binary and numerical columns separately. It would be more convenient if we had just one transformer capable to handle all columns, applying the appropriate transformations to each column. Solution: we can use scikit-learn ColumnTransformer!

In [ ]:
from sklearn.compose import ColumnTransformer

column_transformer = ColumnTransformer(
    (
        ("numerical", num_pipeline, num_feats),
        ("categorical", OneHotEncoder(categories='auto', sparse_output=False).set_output(transform="pandas"), cat_feats),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

column_transformer

In [ ]:
train_data_prepared = column_transformer.fit_transform(train_data)
train_data_prepared

Most of the steps in the pipeline are now chained together. We are just missing the `engineering_features()` transformation step. We can use [`FunctionTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html#sklearn.preprocessing.FunctionTransformer) to convert `engineering_features()` to a scikit-learn's transformer.

### **Exercise:** create a new pipeline chaining the `FeatureEngineeringTransformer` with the `column_transformer` we have defined above and apply the transformation to the original "raw" `train_set` dataset.

Hint: you can also use the `make_pipeline()` utility function to easily create a pipeline.

In [ ]:
# write your solution here:
full_pipeline = ...

If your implementation is correct the cells below will work correctly:

In [ ]:
full_pipeline.fit(train_set.drop(columns=["price"]), train_set["price"])

In [ ]:
train_data_prepared =  full_pipeline.transform(train_set.drop(columns=["price"]))
train_data_prepared

Great job: we now have all our data preparation pipeline encapsuled in the `full_pipeline` object.

Now we can prepare the test set for the final evaluation as well.

*Nota bene:* you must never fit an estimator (predictor or transformer) to the test set. This would mean leaking your test data into the training phase and would invalidate any conclusions on generalisation you may draw from evaluation on the test set. Hence, we must only use the `full_pipeline.transform()` method with our test set.

In [ ]:
test_data_prepared = full_pipeline.transform(test_set.drop(columns=["price"]))

In [ ]:
test_data_prepared.head()

### 7. Save all the prepared data - train and test

Let's add back the "price" column to our prepared datasets.

In [ ]:
train_set_prepared = pd.concat([train_data_prepared, train_set["price"]], axis=1)
train_set_prepared.sample(5, random_state=77)

In [ ]:
test_set_prepared = pd.concat([test_data_prepared, test_set["price"]], axis=1)
test_set_prepared.sample(5, random_state=77)

Let's save the datasets as CSV files in a newly created `prepared` subdirectory of `datasets`:

In [ ]:
os.makedirs("../datasets/prepared/", exist_ok=True)
train_set_prepared.to_csv("../datasets/prepared/kd-housing-train.csv", index=False)
test_set_prepared.to_csv("../datasets/prepared/kd-housing-test.csv", index=False)

In [ ]:
train_set_prepared.info()